# AI Lọc Nội Dung Độc Hại - PhoBERT Production

Hệ thống phát hiện nội dung nhạy cảm tiếng Việt sử dụng PhoBERT.

**Tính năng:**
- Phát hiện 6 loại nội dung: Safe, Toxic, Hate, Violence, NSFW, Suicide
- Ngưỡng thông minh cho từng loại nội dung
- Tối ưu hóa cho production (inference nhanh)

**Mô hình:** vinai/phobert-base-v2 (fine-tuned)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Cài Đặt Thư Viện

In [4]:
!pip install transformers torch pandas -q

print("✓ Đã cài đặt xong các thư viện cần thiết!")

✓ Đã cài đặt xong các thư viện cần thiết!


## 2. Import Thư Viện

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from typing import Dict, Tuple
import warnings

warnings.filterwarnings('ignore')

print("✓ Import thành công!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✓ Import thành công!
PyTorch version: 2.9.0+cu126
CUDA available: True


## 3. Class PhoBERT Moderator (Production-Ready)

In [6]:
class PhoBERTModerator:
    """
    Hệ thống lọc nội dung độc hại tiếng Việt sử dụng PhoBERT
    Tối ưu hóa cho production với biến static và inference nhanh
    """

    # Biến static - khởi tạo 1 lần, tái sử dụng nhiều lần
    SMART_THRESHOLDS = {
        'suicide': 0.35,   # CỰC KỲ NGHIÊM TRỌNG - ngưỡng thấp
        'violence': 0.40,  # Rất nghiêm trọng - đe dọa/khủng bố
        'nsfw': 0.50,      # Nội dung 18+
        'toxic': 0.60,     # Độc hại
        'hate': 0.70,      # Kỳ thị - ngưỡng cao (tránh nhầm tranh luận)
        'safe': 0.30       # An toàn
    }

    LABEL_EMOJI = {
        'safe': '✅',
        'toxic': '⚠️',
        'hate': '🚫',
        'violence': '⚔️',
        'nsfw': '🔞',
        'suicide': '💀'
    }

    CUMULATIVE_THRESHOLD = 0.60  # Ngưỡng tổng điểm tiêu cực

    def __init__(self, model_path: str = '/content/drive/MyDrive/phobert_vietnamese_moderation'):
        """
        Khởi tạo model PhoBERT

        Args:
            model_path: Đường dẫn đến model đã fine-tune
        """
        print("Đang tải PhoBERT model...")

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()

        # Label mapping
        self.id2label = self.model.config.id2label
        self.label2id = self.model.config.label2id

        print(f"✓ Model loaded on {self.device}")
        print(f"✓ Labels: {list(self.id2label.values())}")

    def predict(self, text: str) -> Dict:
        """
        Dự đoán label cho văn bản với ngưỡng thông minh

        Args:
            text: Văn bản cần kiểm tra

        Returns:
            Dict chứa kết quả phân tích
        """
        # Tokenize
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        # Inference
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[0]

        # Tính điểm cho từng label
        label_scores = {self.id2label[i]: float(probs[i]) for i in range(len(probs))}

        # Tính tổng điểm tiêu cực
        negative_labels = ['suicide', 'nsfw', 'violence', 'toxic', 'hate']
        cumulative_negative = sum(label_scores.get(label, 0.0) for label in negative_labels)

        # Áp dụng logic ngưỡng thông minh
        if cumulative_negative > self.CUMULATIVE_THRESHOLD:
            # Tổng điểm tiêu cực cao -> chọn label tiêu cực cao nhất
            negative_scores = {label: label_scores[label] for label in negative_labels
                             if label in label_scores}
            final_label = max(negative_scores.items(), key=lambda x: x[1])[0]
            final_confidence = label_scores[final_label]
        else:
            # Kiểm tra từng label theo thứ tự nghiêm trọng
            final_label = 'safe'
            final_confidence = label_scores.get('safe', 0.0)

            for label in negative_labels:
                if label in label_scores:
                    score = label_scores[label]
                    threshold = self.SMART_THRESHOLDS[label]

                    if score >= threshold:
                        final_label = label
                        final_confidence = score
                        break

        # Đánh giá mức độ rủi ro
        risk_level = self._assess_risk(final_label, final_confidence, cumulative_negative)

        return {
            'text': text,
            'label': final_label,
            'confidence': final_confidence,
            'cumulative_negative': cumulative_negative,
            'all_scores': label_scores,
            'risk_level': risk_level,
            'is_safe': final_label == 'safe'
        }

    def _assess_risk(self, label: str, confidence: float, cumulative: float) -> str:
        """Đánh giá mức độ rủi ro"""
        if label == 'safe':
            return 'no_risk'
        elif label in ['suicide', 'violence'] or cumulative > 0.75:
            return 'high_risk'
        elif confidence > 0.80 or cumulative > 0.65:
            return 'medium_risk'
        else:
            return 'low_risk'

    def print_result(self, result: Dict):
        """In kết quả đẹp mắt"""
        emoji = self.LABEL_EMOJI.get(result['label'], '❓')

        print("=" * 70)
        print(f"Text: {result['text']}")
        print(f"\n{emoji} LABEL: {result['label'].upper()}")
        print(f"Confidence: {result['confidence']:.2%}")
        print(f"Tổng điểm tiêu cực: {result['cumulative_negative']:.2%}")
        print(f"Mức độ rủi ro: {result['risk_level'].upper()}")
        print(f"An toàn: {'CÓ ✓' if result['is_safe'] else 'KHÔNG ✗'}")

        # Hiển thị chi tiết điểm
        print(f"\nChi tiết điểm:")
        sorted_scores = sorted(result['all_scores'].items(), key=lambda x: x[1], reverse=True)
        for label, score in sorted_scores:
            if score > 0.05:
                bar = '█' * int(score * 20)
                print(f"  {label:10s}: {score:.2%} {bar}")
        print("=" * 70)

    def batch_predict(self, texts: list) -> list:
        """Dự đoán cho nhiều văn bản cùng lúc"""
        results = []
        for text in texts:
            results.append(self.predict(text))
        return results


print("✓ Class PhoBERTModerator đã sẵn sàng!")

✓ Class PhoBERTModerator đã sẵn sàng!


## 4. Khởi Tạo Model

In [7]:
# Khởi tạo model (chỉ 1 lần duy nhất)
moderator = PhoBERTModerator(model_path='/content/drive/MyDrive/phobert_vietnamese_moderation')

print("\n✓ Model đã sẵn sàng để sử dụng!")

Đang tải PhoBERT model...
✓ Model loaded on cuda
✓ Labels: ['safe', 'toxic', 'hate', 'violence', 'nsfw', 'suicide']

✓ Model đã sẵn sàng để sử dụng!


## 5. Demo - Test Với Các Câu Tiếng Việt

In [8]:
# Test cases đa dạng
test_cases = [
    # SAFE
    "Hôm nay trời đẹp quá! Mình đi chơi với bạn bè rất vui",
    "Ảnh đẹp quá bạn ơi! Chúc bạn một ngày tốt lành",

    # TOXIC
    "Mày ngu vãi lồn, đồ ngốc",
    "Địt mẹ mày, cút đi",

    # HATE
    "Bọn đồng tính phải chết hết đi",
    "Miền Bắc ngu hơn miền Nam",

    # VIOLENCE
    "Tao sẽ giết mày bây giờ",
    "Đánh chết mày cho biết tay",

    # NSFW
    "Link sex của em đây anh ơi",
    "Khỏa thân show hàng nè",

    # SUICIDE
    "Cuộc sống vô nghĩa, tôi muốn tự tử",
    "Không muốn sống nữa, chết đi cho rồi"
]

print("=" * 70)
print("DEMO - KIỂM TRA NỘI DUNG TIẾNG VIỆT")
print("=" * 70)

for i, text in enumerate(test_cases, 1):
    print(f"\n[Test {i}/{len(test_cases)}]")
    result = moderator.predict(text)
    moderator.print_result(result)
    print()

DEMO - KIỂM TRA NỘI DUNG TIẾNG VIỆT

[Test 1/12]
Text: Hôm nay trời đẹp quá! Mình đi chơi với bạn bè rất vui

✅ LABEL: SAFE
Confidence: 99.60%
Tổng điểm tiêu cực: 0.40%
Mức độ rủi ro: NO_RISK
An toàn: CÓ ✓

Chi tiết điểm:
  safe      : 99.60% ███████████████████


[Test 2/12]
Text: Ảnh đẹp quá bạn ơi! Chúc bạn một ngày tốt lành

✅ LABEL: SAFE
Confidence: 99.61%
Tổng điểm tiêu cực: 0.39%
Mức độ rủi ro: NO_RISK
An toàn: CÓ ✓

Chi tiết điểm:
  safe      : 99.61% ███████████████████


[Test 3/12]
Text: Mày ngu vãi lồn, đồ ngốc

🚫 LABEL: HATE
Confidence: 91.07%
Tổng điểm tiêu cực: 99.35%
Mức độ rủi ro: HIGH_RISK
An toàn: KHÔNG ✗

Chi tiết điểm:
  hate      : 91.07% ██████████████████


[Test 4/12]
Text: Địt mẹ mày, cút đi

🚫 LABEL: HATE
Confidence: 52.22%
Tổng điểm tiêu cực: 99.29%
Mức độ rủi ro: HIGH_RISK
An toàn: KHÔNG ✗

Chi tiết điểm:
  hate      : 52.22% ██████████
  toxic     : 26.02% █████
  violence  : 17.58% ███


[Test 5/12]
Text: Bọn đồng tính phải chết hết đi

🚫 LABEL: HATE
Conf

## 6. Sử Dụng Trong Production

In [9]:
def check_content(text: str) -> bool:
    """
    Hàm đơn giản để kiểm tra nội dung

    Args:
        text: Văn bản cần kiểm tra

    Returns:
        True nếu an toàn, False nếu nguy hiểm
    """
    result = moderator.predict(text)
    return result['is_safe']


def moderate_post(caption: str) -> dict:
    """
    Kiểm tra bài post và đưa ra quyết định

    Args:
        caption: Nội dung bài post

    Returns:
        Dict chứa quyết định và lý do
    """
    result = moderator.predict(caption)

    decision = {
        'allow': result['is_safe'],
        'action': '',
        'reason': '',
        'label': result['label'],
        'confidence': result['confidence']
    }

    if result['is_safe']:
        decision['action'] = 'APPROVE'
        decision['reason'] = 'Nội dung an toàn'
    else:
        risk = result['risk_level']
        if risk == 'high_risk':
            decision['action'] = 'BLOCK'
            decision['reason'] = f'Phát hiện {result["label"]} - Tự động chặn'
        elif risk == 'medium_risk':
            decision['action'] = 'REVIEW'
            decision['reason'] = f'Nghi ngờ {result["label"]} - Cần xem xét'
        else:
            decision['action'] = 'WARNING'
            decision['reason'] = f'Cảnh báo {result["label"]} - Nhắc nhở người dùng'

    return decision


# Demo sử dụng
print("=" * 70)
print("VÍ DỤ SỬ DỤNG TRONG PRODUCTION")
print("=" * 70)

test_posts = [
    "Hôm nay đi du lịch Đà Lạt, view đẹp lắm!",
    "Bọn mày là lũ ngu ngốc, đáng chết!",
    "Cuộc sống quá khó khăn, tôi muốn tự tử"
]

for i, post in enumerate(test_posts, 1):
    print(f"\n--- Bài Post {i} ---")
    print(f"Caption: {post}")

    decision = moderate_post(post)

    print(f"Quyết định: {decision['action']}")
    print(f"Lý do: {decision['reason']}")
    print(f"Label: {decision['label']} ({decision['confidence']:.2%})")
    print("-" * 70)

VÍ DỤ SỬ DỤNG TRONG PRODUCTION

--- Bài Post 1 ---
Caption: Hôm nay đi du lịch Đà Lạt, view đẹp lắm!
Quyết định: APPROVE
Lý do: Nội dung an toàn
Label: safe (99.57%)
----------------------------------------------------------------------

--- Bài Post 2 ---
Caption: Bọn mày là lũ ngu ngốc, đáng chết!
Quyết định: BLOCK
Lý do: Phát hiện hate - Tự động chặn
Label: hate (90.70%)
----------------------------------------------------------------------

--- Bài Post 3 ---
Caption: Cuộc sống quá khó khăn, tôi muốn tự tử
Quyết định: BLOCK
Lý do: Phát hiện suicide - Tự động chặn
Label: suicide (98.72%)
----------------------------------------------------------------------


## 8. Demo Batch Prediction (Xử Lý Nhiều Văn Bản)

In [10]:
# Xử lý nhiều comment cùng lúc
comments = [
    "Ảnh đẹp quá bạn ơi!",
    "Mày là thằng ngu",
    "Hôm nay vui ghê!",
    "Bọn LGBT phải chết hết",
    "Món này ngon quá!"
]

print("=" * 70)
print("BATCH PREDICTION - XỬ LÝ NHIỀU COMMENT")
print("=" * 70)

results = moderator.batch_predict(comments)

# Tổng hợp kết quả
safe_count = sum(1 for r in results if r['is_safe'])
unsafe_count = len(results) - safe_count

print(f"\nTổng số comment: {len(results)}")
print(f"✓ An toàn: {safe_count}")
print(f"✗ Nguy hiểm: {unsafe_count}")

print(f"\nChi tiết:\n")
for i, result in enumerate(results, 1):
    emoji = moderator.LABEL_EMOJI.get(result['label'], '❓')
    status = "✓ AN TOÀN" if result['is_safe'] else "✗ NGUY HIỂM"

    print(f"{i}. {emoji} {result['text'][:50]}...")
    print(f"   {status} | Label: {result['label'].upper()} ({result['confidence']:.0%})")
    print()

print("=" * 70)

BATCH PREDICTION - XỬ LÝ NHIỀU COMMENT

Tổng số comment: 5
✓ An toàn: 3
✗ Nguy hiểm: 2

Chi tiết:

1. ✅ Ảnh đẹp quá bạn ơi!...
   ✓ AN TOÀN | Label: SAFE (100%)

2. 🚫 Mày là thằng ngu...
   ✗ NGUY HIỂM | Label: HATE (58%)

3. ✅ Hôm nay vui ghê!...
   ✓ AN TOÀN | Label: SAFE (100%)

4. 🚫 Bọn LGBT phải chết hết...
   ✗ NGUY HIỂM | Label: HATE (83%)

5. ✅ Món này ngon quá!...
   ✓ AN TOÀN | Label: SAFE (100%)

